# Initial Steps

- `mkdir -p ./DWREApiDoc.docset/Contents/Resources/Documents`
- copy scriptapi/api to Documents/api and pipeletapi/api to Documents/pipelet (i.e. rename folder to pipelet)
- Run this script

In [49]:
from pyquery import PyQuery as pq
import sqlite3
import os.path
import shutil
import markdown
from markdown.extensions.tables import TableExtension
from markdown.extensions.fenced_code import FencedCodeExtension
from markdown.extensions.codehilite import CodeHiliteExtension
from jinja2 import Template

In [50]:
PLIST = """<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
<key>CFBundleIdentifier</key>
<string>oms</string>
<key>CFBundleName</key>
<string>OMS</string>
<key>DocSetPlatformFamily</key>
<string>oms</string>
<key>isDashDocset</key>
<true/>
</dict>
</plist>"""

# Init

In [51]:
if not os.path.exists('./OrderManagement.docset/Contents/Resources/Documents/'):
    os.makedirs('./OrderManagement.docset/Contents/Resources/Documents/')
    
shutil.rmtree("./OrderManagement.docset/Contents/")

if not os.path.exists('./OrderManagement.docset/Contents/Resources/Documents/'):
    os.makedirs('./OrderManagement.docset/Contents/Resources/Documents/')

shutil.copy('./dwredocs/code.css', './OrderManagement.docset/Contents/Resources/Documents/code.css')

with open('./OrderManagement.docset/Contents/Info.plist', 'w') as f:
    f.write(PLIST)
    
conn = sqlite3.connect('./OrderManagement.docset/Contents/Resources/docSet.dsidx')
c = conn.cursor()
c.execute("CREATE TABLE IF NOT EXISTS searchIndex(id INTEGER PRIMARY KEY, name TEXT, type TEXT, path TEXT);")
c.execute("CREATE UNIQUE INDEX IF NOT EXISTS anchor ON searchIndex (name, type, path);")
conn.commit()

# Script API

In [52]:
if os.path.exists("./OrderManagement.docset/Contents/Resources/Documents/api"):
    shutil.rmtree("./OrderManagement.docset/Contents/Resources/Documents/api")
shutil.copytree("./oms/documentation.demandware.com/API1/topic/com.demandware.dochelp/DWAPP-17.7-API-doc/scriptapi/html/api/", "./OrderManagement.docset/Contents/Resources/Documents/api")

In [53]:
with open("./OrderManagement.docset/Contents/Resources/Documents/api/classList.html", "r") as f:
    d = pq(f.read())

c = conn.cursor()

for link in d('table a'):

    title = link.attrib["title"]
    name = link.text
    path = link.attrib["href"]
    if "dw.om" not in title:
        continue
    print(name, title, path)
    
    c.execute("INSERT OR IGNORE INTO searchIndex(name, type, path) VALUES ('%s', 'Class', '%s');" % 
              (name, "api/%s" % path))
    
conn.commit()
def test():
    pass

('AddressDetail', 'Class in dw.om.common', 'class_dw_om_common_AddressDetail.html')
('AddressSync', 'Class in dw.om.common', 'class_dw_om_common_AddressSync.html')
('AdviceInstructionFulfillWithServiceProviderResponse', 'Class in dw.om.shippingorders', 'class_dw_om_shippingorders_AdviceInstructionFulfillWithServiceProviderResponse.html')
('AdviceInstructionGuidResponse', 'Class in dw.om.inventory', 'class_dw_om_inventory_AdviceInstructionGuidResponse.html')
('AdviceInstructionOrderAllocateResponse', 'Class in dw.om.orders', 'class_dw_om_orders_AdviceInstructionOrderAllocateResponse.html')
('AdviceInstructionOrderTransactionExecuteResult', 'Class in dw.om.orders', 'class_dw_om_orders_AdviceInstructionOrderTransactionExecuteResult.html')
('AdviceInstructionOrderTransactionResponse', 'Class in dw.om.orders', 'class_dw_om_orders_AdviceInstructionOrderTransactionResponse.html')
('AdviceInstructionResponse', 'Class in dw.om.customers', 'class_dw_om_customers_AdviceInstructionResponse.html')


In [54]:
conn.close()